In [1]:
pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 42 kB/s 
     |████████████████████████████████| 199 kB 47.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=defe006c97a113dd4071a910d9d8466b14acde61d887da2a7908b2896f1b30d0
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
import pyspark

In [4]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Practical_Day_1_DF_SQL_ML").getOrCreate()

In [7]:
data = spark.read.csv("PatientInfo.csv", header=True, inferSchema=True)
data.show()

+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------+--------+
|patient_id|   sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|     confirmed_date|      released_date|deceased_date|   state|
+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|2020-01-23 00:00:00|2020-02-05 00:00:00|         null|released|
|1000000002|  male|30s|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       null|            31|              null|2020-01-30 00:00:00|2020-03-02 00:00:00|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|   Jongno-gu|contact with patient| 20020

In [8]:
data.printSchema()

root
 |-- patient_id: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: timestamp (nullable = true)
 |-- released_date: timestamp (nullable = true)
 |-- deceased_date: timestamp (nullable = true)
 |-- state: string (nullable = true)



In [9]:
data.describe().show()

+-------+--------------------+------+----+----------+--------+--------------+--------------------+--------------------+--------------------+------------------+--------+
|summary|          patient_id|   sex| age|   country|province|          city|      infection_case|         infected_by|      contact_number|symptom_onset_date|   state|
+-------+--------------------+------+----+----------+--------+--------------+--------------------+--------------------+--------------------+------------------+--------+
|  count|                5165|  4043|3785|      5165|    5165|          5071|                4246|                1346|                 791|               690|    5165|
|   mean|2.8636345618679576E9|  null|null|      null|    null|          null|                null|2.2845944015643125E9|1.6772572523506988E7|              null|    null|
| stddev| 2.074210725277473E9|  null|null|      null|    null|          null|                null|1.5265072953383324E9| 3.093097580985502E8|              n

In [10]:
data.groupBy("state").count().show()

+--------+-----+
|   state|count|
+--------+-----+
|isolated| 2158|
|released| 2929|
|deceased|   78|
+--------+-----+



In [11]:
data.select([count(when(col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+
|patient_id| sex| age|country|province|city|infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|state|
+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+
|         0|1122|1380|      0|       0|  94|           919|       3819|          4374|              4475|             3|         3578|         5099|    0|
+----------+----+----+-------+--------+----+--------------+-----------+--------------+------------------+--------------+-------------+-------------+-----+



In [12]:
datacopy = data.withColumn("deceased_date", coalesce(data.deceased_date, data.released_date))

In [13]:
datacopy = data.withColumn("deceased_date", coalesce(data.deceased_date, data.released_date))

In [14]:
#Add a column named no_days which is difference between the deceased_date and the confirmed_date then show the top 5 rows. Print the schema.
#Hint: You need to typecast these columns as date first

sdata = datacopy.withColumn("no_days", datediff(datacopy.deceased_date, datacopy.confirmed_date).cast("int"))

In [15]:
#### Add a is_male column if male then it should yield true, else then False
sdata.select(col("sex") == "male").show()

+------------+
|(sex = male)|
+------------+
|        true|
|        true|
|        true|
|        true|
|       false|
|       false|
|        true|
|        true|
|        true|
|       false|
|       false|
|        true|
|        true|
|       false|
|        true|
|        true|
|        true|
|        true|
|       false|
|       false|
+------------+
only showing top 20 rows



In [16]:
wismale = sdata.withColumn("is_male", col("sex") == "male")
wismale.show()

+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------------+--------+-------+-------+
|patient_id|   sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|     confirmed_date|      released_date|      deceased_date|   state|no_days|is_male|
+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------------+--------+-------+-------+
|1000000001|  male|50s|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|2020-01-23 00:00:00|2020-02-05 00:00:00|2020-02-05 00:00:00|released|     13|   true|
|1000000002|  male|30s|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       null|            31|              null|2020-01-30 00:00:00|2020-03-02 00:00:00|2020-03-02 00:00

In [17]:
#Add a is_dead column if patient state is not released then it should yield true, else then False
isdead = wismale.withColumn("is_dead", col("state") != "released")
isdead.show()

+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------------+--------+-------+-------+-------+
|patient_id|   sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|     confirmed_date|      released_date|      deceased_date|   state|no_days|is_male|is_dead|
+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------------+--------+-------+-------+-------+
|1000000001|  male|50s|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|2020-01-23 00:00:00|2020-02-05 00:00:00|2020-02-05 00:00:00|released|     13|   true|  false|
|1000000002|  male|30s|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       null|            31|              null|2020-01-30 00:00:00|2020

In [19]:
# Change the ages to bins from 10s, 0s, 10s, 20s,.etc to 0,10, 20
@udf
def age_bins(age):
    if age is None or age is None:
        return age
    age = age.split("s")[0]
    return age
afterages = isdead.withColumn("age", age_bins(isdead.age))
afterages.show(5)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------------+--------+-------+-------+-------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|     confirmed_date|      released_date|      deceased_date|   state|no_days|is_male|is_dead|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------------+--------+-------+-------+-------+
|1000000001|  male| 50|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|2020-01-23 00:00:00|2020-02-05 00:00:00|2020-02-05 00:00:00|released|     13|   true|  false|
|1000000002|  male| 30|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|2020-01-30 00:00:00|2020-03-0

In [20]:
#Change age, and no_days to be typecasted as Double
datap = afterages.withColumn("age", col("age").cast("double")).withColumn("no_days", col("no_days").cast("double"))
datap.show(5)

+----------+------+----+-------+--------+-----------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------------+--------+-------+-------+-------+
|patient_id|   sex| age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|     confirmed_date|      released_date|      deceased_date|   state|no_days|is_male|is_dead|
+----------+------+----+-------+--------+-----------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------------+--------+-------+-------+-------+
|1000000001|  male|50.0|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|2020-01-23 00:00:00|2020-02-05 00:00:00|2020-02-05 00:00:00|released|   13.0|   true|  false|
|1000000002|  male|30.0|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|2020-01-30 00:00:00|2020

In [21]:
#Drop the columns
ddata = datap.drop("patient_id", "sex", "infected_by", "contact_number","released_date","state","symptom_onset_date","confirmed_date","deceased_date","country","no_days", "city","infection_case")
ddata.show(5)

+----+--------+-------+-------+
| age|province|is_male|is_dead|
+----+--------+-------+-------+
|50.0|   Seoul|   true|  false|
|30.0|   Seoul|   true|  false|
|50.0|   Seoul|   true|  false|
|20.0|   Seoul|   true|  false|
|20.0|   Seoul|  false|  false|
+----+--------+-------+-------+
only showing top 5 rows



In [22]:
#Recount the number of nulls now
ddata.select([count(when(col(c).isNull(), c)).alias(c) for c in ddata.columns]).show()

+----+--------+-------+-------+
| age|province|is_male|is_dead|
+----+--------+-------+-------+
|1380|       0|   1122|      0|
+----+--------+-------+-------+



In [23]:
#Now do the same but using SQL select statement
#From the original Patient DataFrame, Create a temporary view (table).
SQLdata = data.createOrReplaceTempView("SQLdata")


In [24]:
#Use SELECT statement to select all columns from the dataframe and show the output.
Selectdata = spark.sql("SELECT * FROM SQLdata")

In [25]:
#Using SQL commands, limit the output to only 5 rows
Selectdata.show()

+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------+--------+
|patient_id|   sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|     confirmed_date|      released_date|deceased_date|   state|
+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|2020-01-23 00:00:00|2020-02-05 00:00:00|         null|released|
|1000000002|  male|30s|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       null|            31|              null|2020-01-30 00:00:00|2020-03-02 00:00:00|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|   Jongno-gu|contact with patient| 20020

In [26]:
#Select the count of males and females in the dataset
spark.sql("SELECT sex, COUNT(patient_id)\
            from SQLdata\
            group by sex").show()

+------+-----------------+
|   sex|count(patient_id)|
+------+-----------------+
|  null|             1122|
|female|             2218|
|  male|             1825|
+------+-----------------+



In [27]:
#How many people did survive, and how many didn't?
spark.sql("SELECT state ,count(patient_id)\
            from SQLdata\
            group by 1").show()

+--------+-----------------+
|   state|count(patient_id)|
+--------+-----------------+
|isolated|             2158|
|released|             2929|
|deceased|               78|
+--------+-----------------+



In [28]:
ppdata = spark.sql("SELECT sex, SUBSTRING(age, 1, 2) as age, province, state\
            from SQLdata")
ppdata.show()

+------+---+--------+--------+
|   sex|age|province|   state|
+------+---+--------+--------+
|  male| 50|   Seoul|released|
|  male| 30|   Seoul|released|
|  male| 50|   Seoul|released|
|  male| 20|   Seoul|released|
|female| 20|   Seoul|released|
|female| 50|   Seoul|released|
|  male| 20|   Seoul|released|
|  male| 20|   Seoul|released|
|  male| 30|   Seoul|released|
|female| 60|   Seoul|released|
|female| 50|   Seoul|released|
|  male| 20|   Seoul|released|
|  male| 80|   Seoul|deceased|
|female| 60|   Seoul|released|
|  male| 70|   Seoul|released|
|  male| 70|   Seoul|released|
|  male| 70|   Seoul|released|
|  male| 20|   Seoul|released|
|female| 70|   Seoul|released|
|female| 70|   Seoul|released|
+------+---+--------+--------+
only showing top 20 rows



In [29]:
#Machine learning
@udf
def boolean_to_int(boolean_):
    if boolean_ is None:
        return 0
    if boolean_ is True:
        return 1
    else:
        return 0

In [30]:
MLdata = ddata.withColumn("is_male", boolean_to_int(ddata.is_male).cast("int"))
MLdata1 = MLdata.withColumn("is_dead", boolean_to_int(MLdata.is_dead).cast("int"))
MLdata1.dtypes

[('age', 'double'),
 ('province', 'string'),
 ('is_male', 'int'),
 ('is_dead', 'int')]

In [31]:
MLdata1.select([count(when(col(c).isNull(), c)).alias(c) for c in MLdata1.columns]).show()

+----+--------+-------+-------+
| age|province|is_male|is_dead|
+----+--------+-------+-------+
|1380|       0|      0|      0|
+----+--------+-------+-------+



In [33]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, Imputer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [34]:
si = StringIndexer(inputCol="province", outputCol="province_index")
ohe = OneHotEncoder(inputCol="is_male", outputCol="is_male_vec")
imputer = Imputer(inputCol= "age", outputCol="age" )
features = ["age","province_index", "is_male_vec"]
va = VectorAssembler(inputCols=features, outputCol="vec_features")
lr = LinearRegression(featuresCol="vec_features", labelCol="is_dead")
dt = RandomForestClassifier(featuresCol="vec_features", labelCol="is_dead",numTrees=5, maxDepth=5)  

In [35]:
training, testing = MLdata1.randomSplit([0.8, 0.2])

In [36]:
pipe = Pipeline(stages=[si, ohe, imputer,va, lr])
spipe = Pipeline(stages=[si, ohe, imputer,va, dt])
model = pipe.fit(training)
smodel = spipe.fit(testing)
fmpred= model.transform(testing)
smpred= smodel.transform(testing)
fmpred.describe().show()
smpred.describe().show()

+-------+------------------+--------+------------------+-------------------+------------------+-------------------+
|summary|               age|province|           is_male|            is_dead|    province_index|         prediction|
+-------+------------------+--------+------------------+-------------------+------------------+-------------------+
|  count|              1017|    1017|              1017|               1017|              1017|               1017|
|   mean| 40.77005050271039|    null|0.3392330383480826| 0.4326450344149459|2.5191740412979353|0.43588314114835613|
| stddev|17.669616275873672|    null|0.4736819693013992|0.49568629833366623| 3.186443372220012|0.05998848529120532|
|    min|               0.0|   Busan|                 0|                  0|               0.0| 0.2697320009238058|
|    max|              90.0|   Ulsan|                 1|                  1|              16.0| 0.6400247775816934|
+-------+------------------+--------+------------------+----------------

In [37]:
ev= RegressionEvaluator(predictionCol="prediction", labelCol="is_dead", metricName="rmse")
sev = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="is_dead")
ev.evaluate(fmpred)

0.49581117212511705

In [38]:
sev.evaluate(smpred)

0.817086812667402